In [35]:
# Environment Variables: 

YES = 1
NO = 0

DATA_DIR = '/home/pujan/Research/RHEED/Data/' # Change to your DATA PATH
using_GPU = YES

In [ ]:
# Imports

import os
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool
import h5py
from sklearn.preprocessing import StandardScaler
from scipy.ndimage import median_filter
from scipy.optimize import least_squares
from dask import delayed, compute
from dask.distributed import Client
from tqdm import tqdm
import tensorflow as tf
from qkeras import *

%matplotlib inline
output_scaler = StandardScaler()
if(using_GPU):
    print(tf.config.list_physical_devices('GPU'))

In [ ]:
# Read H5 Data File:

RHEED_data_file = DATA_DIR + 'RHEED_4848_test6.h5'
spot = 'spot_2'
h5 = h5py.File(RHEED_data_file, 'r')

raw_data = []
for growth in h5.keys():
    raw_data.extend(h5[growth][spot])
raw_data = np.array(raw_data).astype(np.float32)
raw_data = np.expand_dims(raw_data, axis=-1).astype(np.float32) # if (batch_size, height, width, channels)

print(f'[Raw Images Shape]: {raw_data.shape}')

In [ ]:
# Normalize w/ image max

normalized_images = []
normalized_factor = []
for image in tqdm(raw_data):
    normalized_images.append(image / np.max(image))
    normalized_factor.append(np.max(image))
normalized_images = np.array(normalized_images).astype(np.float32)
normalized_factor = np.array(normalized_factor).astype(np.float32)

print(f'[Normalized Images Shape]: {normalized_images.shape}')

In [ ]:
# Estimate Labels:
load_labels = YES # (Takes <1 min to load, ~40 mins to generate)

# Import From File

if load_labels:
    RHEED_LABEL_FILE = DATA_DIR + 'Estimated_Labels_PP.npy'
    estimated_labels = np.load(RHEED_LABEL_FILE)

# Generate :(
else:
    pass

print(f'[Estimated Labels Shape]: {estimated_labels.shape}')

In [ ]:
# Create DataSet:
batch_size = 1000
dataset_arr = normalized_images

dataset = tf.data.Dataset.from_tensor_slices(dataset_arr)
dataset = dataset.shuffle(dataset_arr.shape[0], reshuffle_each_iteration=True)
dataset = dataset.batch(batch_size)

In [32]:
# Gaussian Functions: (TENSORFLOW)
print_example_guassian = NO
print_example_loss = NO

# mean_x, mean_y, cov_x, cov_y, theta
def generate_guassian(batch, image_shape):
    batch_size = batch.shape[0]
    batch = tf.expand_dims(tf.expand_dims(batch, axis=-1), axis=-1)
    x0, y0, sigma_x, sigma_y, theta = tf.cast(tf.unstack(batch, axis=-3), tf.float32)
    
    x_range = tf.range(start=0, limit=image_shape[0], delta=1)
    y_range = tf.range(start=0, limit=image_shape[1], delta=1)
    X_coord, Y_coord = tf.meshgrid(x_range, y_range, indexing='xy')
    X_coord = tf.cast(tf.expand_dims(X_coord, axis=0), tf.float32)
    Y_coord = tf.cast(tf.expand_dims(Y_coord, axis=0), tf.float32)
    
    X_coord = tf.tile(X_coord, [batch_size, 1, 1])
    Y_coord = tf.tile(Y_coord, [batch_size, 1, 1])
    
    a = tf.math.pow(tf.math.cos(theta), 2) / (2 * tf.math.pow(sigma_x, 2)) + tf.math.pow(tf.math.sin(theta), 2) / (2 * tf.math.pow(sigma_y, 2))
    b = -1 * tf.math.sin(theta) * tf.math.cos(theta) / (2 * tf.math.pow(sigma_x, 2)) + tf.math.sin(theta) * tf.math.cos(theta) / (2 * tf.math.pow(sigma_y, 2))
    c = tf.math.pow(tf.math.sin(theta), 2) / (2 * tf.math.pow(sigma_x, 2)) + tf.math.pow(tf.math.cos(theta), 2) / (2 * tf.math.pow(sigma_y, 2))

    img = tf.exp(-1 * (a * (X_coord - x0) ** 2 + 2 * b * (X_coord - x0) * (Y_coord - y0) + c * (Y_coord - y0) ** 2))

    return tf.expand_dims(img, axis=-1) # if (batch_size, height, width, channels)
    return tf.expand_dims(img, axis=1)  # if (batch_size, channels, height, width)

def custom_weighted_mse_loss(I, J, n):
  W = tf.pow(I, n)

  squared_diffs = tf.pow(I - J, 2)

  weighted_squared_diffs = W * squared_diffs

  loss = tf.reduce_mean(weighted_squared_diffs)

  return loss

if print_example_guassian:
    image_shape = (48, 48)
    batch = tf.convert_to_tensor([
        [21.8558168, 24.50041009, 10.31268177, 9.1700225, 0.72681534]
        , [21.76068143, 24.37956637, 10.30043488, 9.15426013, 0.72655111]
        , [21.72363929, 24.31050759, 10.33800891, 9.18570812, 0.72644599]
        , [21.72777699, 24.29306623, 10.30178808, 9.14728058, 0.72610718]
        , [21.79849472, 24.34649405, 10.32683150, 9.16259293, 0.72573213]
    ])
    generated_imgs = generate_guassian(batch, image_shape)
    plt.imshow(tf.squeeze(generated_imgs[0]))
    plt.show()

if print_example_loss:
    I = tf.random.normal((5, 1, 48, 48))
    J = tf.random.normal((5, 1, 48, 48))
    n = 2
    loss = custom_weighted_mse_loss(I, J, n)
    print("[Custom Weighted MSE Loss]:", loss.numpy())
    

In [33]:
model = tf.keras.Sequential(
    [
        # Quantized Conv2D Layer
        QConv2DBatchnorm(
            filters=6, kernel_size=5, strides=1, padding='valid',
            kernel_quantizer="quantized_bits(8, 0, 1)",  # 8-bit weights
            bias_quantizer="quantized_bits(8, 0, 1)",  # 8-bit biases
        ),
        # Quantized Activation
        QActivation("quantized_relu(8, 0)"),
        tf.keras.layers.MaxPool2D(pool_size=4, strides=4),

        # Second Quantized Conv2D Layer
        QConv2DBatchnorm(
            filters=16, kernel_size=5, strides=1, padding='valid',
            kernel_quantizer="quantized_bits(8, 0, 1)",  # 8-bit weights
            bias_quantizer="quantized_bits(8, 0, 1)",  # 8-bit biases
        ),
        # Quantized Activation
        QActivation("quantized_relu(8, 0)"),
        tf.keras.layers.MaxPool2D(pool_size=2, strides=2),

        # Flatten for Dense Layers
        tf.keras.layers.Flatten(),

        # First Quantized Dense Layer
        QDense(
            units=98,
            kernel_quantizer="quantized_bits(8, 0, 1)",  # 8-bit weights
            bias_quantizer="quantized_bits(8, 0, 1)",  # 8-bit biases
        ),
        QActivation("quantized_relu(8, 0)"),

        # Second Quantized Dense Layer
        QDense(
            units=52,
            kernel_quantizer="quantized_bits(8, 0, 1)",  # 8-bit weights
            bias_quantizer="quantized_bits(8, 0, 1)",  # 8-bit biases
        ),
        QActivation("quantized_relu(8, 0)"),

        # Final Output Layer
        QDense(
            units=5,
            kernel_quantizer="quantized_bits(8, 0, 1)",  # 8-bit weights
            bias_quantizer="quantized_bits(8, 0, 1)",  # 8-bit biases
        )
    ]
)

# Compile the quantization-aware model
model.compile(optimizer='adam', loss=custom_weighted_mse_loss)

In [ ]:
# Training Loop
train_model = YES
save_model = YES
load_model = NO

if train_model:
    best_loss = float('inf')
    num_epochs = 200
    lr = 0.0001
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
    n = 1

    scale = tf.constant([48, 48, 24, 24, np.pi/2])
    for epoch in range(num_epochs):
        running_loss = 0.0

        if epoch % 10 == 0:
            n += 0.1

        for image_batch in tqdm(dataset): 
            with tf.GradientTape() as tape:
                embedding = model(image_batch)
                unscaled_param = tf.constant(embedding * scale)
                final = generate_guassian(unscaled_param, (48, 48))
                loss = custom_weighted_mse_loss(image_batch, final, n)
            grads = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))

            running_loss += loss.numpy()
        average_loss = running_loss / len(dataset)
        print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {average_loss}")

if save_model:
    model.save(f'{DATA_DIR}/Models/[0,8]Gaussian.keras')

if load_model:
    model = tf.keras.models.load_model("Gaussian_Model.keras")

model.summary()